In [2]:
'''
This is a sandbox to work on proving that we can get digital 
sound out of scans of an optical track. If I can manually 
get it to work here, I can then take this PoC and turn it into
a full application.
'''
import cv2
import os
import matplotlib.pyplot as plt
import math
import numpy as np
import wave
import struct
import natsort
from scipy.io import wavfile
from scipy.signal import butter, lfilter

In [3]:
imagefiles = natsort.natsorted(os.listdir("input\\fulltest\."))
print(len(imagefiles))
DURRATION = len(imagefiles) / 24
CUTTOFF_FREQUENCY = 16000
S_INT16_MAX = 32767


288


In [4]:
images = []
cutoff = 0
for filename in imagefiles:
    # if cutoff > 1:
    #     break

    img = cv2.imread("input\\fulltest\\" + filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Default reads in as BGR, we want this as black and white
    img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    
    # # Adjusts the brightness by adding 10 to each pixel value 
    # brightness = 5
    # # Adjusts the contrast by scaling the pixel values by 2.3 
    # contrast = 3 
    # img = cv2.addWeighted(img, contrast, np.zeros(img.shape, img.dtype), 0, brightness) 

    # Create the sharpening kernel
    kernel = np.array([[-1,-1,-1], 
                       [-1,9,-1], 
                       [-1,-1,-1]])
    # Apply the sharpening kernel to the image using filter2D
    img = cv2.filter2D(img, -1, kernel)

    height,width = img.shape
    #img = img[700:height-1000,400:width-2000]

    # This is to trim this exact scan images so that the end of the frames ~perfectly go into the nex
    img = img[965:height-1255,500:width-1000]
    
    # Rotate back
    # img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)

    # Make pixel 1 or 0, "thresholding". This is how we make the optical track either black or white
    ret,img = cv2.threshold(img,128,255,cv2.THRESH_BINARY) # invert so white = 255, black = 0

    images.append(img)
    cutoff = cutoff + 1
num_images = len(images)

images = images[::-1] # Reverse the list order to get the expected order of the optical tracks


# plt.figure(figsize=[30,20])
# for i in range(0, num_images):
#     plt.subplot(num_images, 1, i+1)
#     plt.axis('off')
#     plt.imshow(images[i], cmap='gray') # matplot reads the channels as RGB, so we need to tell it this is actually grey or we get weird channel issues


## Visualize the crop is correct by concatinating the two images

In [23]:
# Concatinated image to show merge visually
# merged_images = cv2.vconcat(images)
# plt.figure(figsize=[30,10])
# plt.subplot(1, 1, 1)
# plt.axis('off')
# plt.imshow(merged_images, cmap='gray')
# cv2.imwrite("mergedDemo.jpeg",merged_images)

## Get just the soundtrack, get mean values per row and convert to sint_16

In [ ]:
### Another Method, not mean but only look at white pixels

sound_only_imgs = images.copy()
mean_values_arr = []

for i in range(0, num_images):
    current_img = sound_only_imgs[i]
    height,width = current_img.shape
    sound_only = current_img[0:height,75:width-1380]
    sound_only_imgs[i] = sound_only

    # View what we did
    # plt.subplot(num_images, 1, i+1)
    # plt.axis('off')
    # plt.imshow(sound_only_imgs[i], cmap='gray') # matplot reads the channels as RGB, so we need to tell it this is actually grey or we get weird channel issues

    np.set_printoptions(threshold=np.inf)
    
    # Get mean value per row (0-255)
    mean_values = np.mean(sound_only, axis=1)

    # Need to get values 0-255 to 0-1 (1 = white, 0 = black)
    mean_values = mean_values / 255.0
    
    # Now get the converted values to sint_16 values (–32768 to 32767)
    # Ill make this cleaner and more constanants in the future
    mean_values = mean_values * S_INT16_MAX
    mean_values = mean_values.astype(int)
    mean_values = mean_values - (int(S_INT16_MAX // 2))
    mean_values_arr.append(mean_values)
    print(len(mean_values))
    cv2.imwrite(f"output\\output{ i }.jpeg",sound_only)

# For some reason we are reversed
mean_values_arr = mean_values_arr[::-1]

## Different Method, not mean but just count all white

In [31]:
sound_only_imgs = images.copy()
count_of_white_pixels_arr = []

temp_count = 0

for i in range(0, num_images):
    current_img = sound_only_imgs[i]
    height,width = current_img.shape
    sound_only = current_img[0:height,65:width-1340]
    sound_only_imgs[i] = sound_only

    # View what we did
    # plt.subplot(num_images, 1, i+1)
    # plt.axis('off')
    # plt.imshow(sound_only_imgs[i], cmap='gray') # matplot reads the channels as RGB, so we need to tell it this is actually grey or we get weird channel issues

    np.set_printoptions(threshold=np.inf)
    
    # 
    count_of_white_pixels = np.sum(sound_only < 255, axis=1)
    # print(count_of_white_pixels)
    count_of_white_pixels_arr.append(count_of_white_pixels)

    # if temp_count > 2:
    #     break

    temp_count+=1
    cv2.imwrite(f"output\\output{ i }.jpeg",sound_only)

count_of_white_pixels_arr = np.array(count_of_white_pixels_arr)
print(count_of_white_pixels_arr)

# Now get max and min
flattened_arr = count_of_white_pixels_arr.flatten()

# Get the highest value
highest_value = np.max(flattened_arr)

# Get the lowest value
lowest_value = np.min(flattened_arr)

print("Highest value:", highest_value)
print("Lowest value:", lowest_value)

# Highest becomes our top, lowest becomes the bottom, aka difference between the two we can use
diff = highest_value - lowest_value

# Now we divide each row value by this diff
count_of_white_pixels_arr = count_of_white_pixels_arr / diff

count_of_white_pixels_arr = count_of_white_pixels_arr * S_INT16_MAX
count_of_white_pixels_arr = count_of_white_pixels_arr.astype(int)
count_of_white_pixels_arr = count_of_white_pixels_arr - (int(S_INT16_MAX // 2))

count_of_white_pixels_arr = count_of_white_pixels_arr.tolist()

print(len(count_of_white_pixels_arr))
cv2.imwrite(f"output\\output{ i }.jpeg",sound_only)


# For some reason we are reversed
mean_values_arr = count_of_white_pixels_arr[::-1]

# print(mean_values_arr)

[[ 81  79  78  75  76  76  75  75  75  76  76  77  77  78  78  80  81  81
   80  81  79  80  79  81  81  80  79  81  79  81  78  80  80  80  81  82
   80  81  82  81  82  81  81  81  82  82  83  83  84  85  86  87  88  89
   90  92  93  94  94  94  94  95  96  97  98 100 101 100 102  98  95  92
   92  91  92  90  91  90  90  86  86  81  79  79  76  75  74  73  72  72
   72  72  72  72  73  72  72  72  71  70  70  70  70  71  71  72  71  70
   70  71  71  73  73  75  76  77  78  79  80  82  80  81  82  82  82  83
   83  84  85  85  86  87  86  87  88  91  91  91  92  91  92  91  92  92
   91  93  92  92  93  95  94  93  94  94  93  93  93  94  93  93  93  95
   95  96  97  98  97  97  97  96  96  97  97  97  97  98  99  99 100 100
   98 100  99  99  97  95  95  93  91  90  90  90  90  89  88  88  87  84
   83  81  78  77  78  78  76  77  77  75  74  73  72  72  72  73  74  75
   77  77  78  79  81  81  82  83  83  80  81  80  78  76  75  77  76  76
   78  79  79  77  75  74  76  76  76 

## Write to a WAV file

In [32]:
sample_rate = 1836 * 24 # Currently making the sample rate the height of each image times the amount of frames in a second
print(f"Sample rate: { sample_rate }")
duration = DURRATION # Length in seconds
num_channels = 1
bit_depth = 16 # sint_16 type
num_frames = int(sample_rate * duration) # amount of frames in a second
samp_per_frame = 1836
raw_output_file = 'output.wav'
filtered_output_file = 'outputFilter.wav'

with wave.open(raw_output_file, 'wb') as output_file:
    output_file.setnchannels(num_channels)
    output_file.setsampwidth(bit_depth // 8)
    output_file.setframerate(sample_rate)
    output_file.setnframes(num_frames)
    output_file.setcomptype('NONE', 'Not Compressed')

    data = b''
    for img_id in range(0,len(mean_values_arr)):
        current_mean_values = mean_values_arr[img_id]

        for i in range(0,samp_per_frame):
            sample = current_mean_values[i]
            packed_sample = struct.pack('<h', sample)
            data += packed_sample
    output_file.writeframesraw(data)



Sample rate: 44064


### Filtering frequenices out of the audio

In [58]:
def high_pass_filter(data, cutoff_frequency, sample_rate, order=5):
    b, a = butter(order, cutoff_frequency / (sample_rate / 2), btype='high')
    filtered_data = lfilter(b, a, data)
    return filtered_data

def low_pass_filter(data, cutoff_frequency, sample_rate, order=5):
    b, a = butter(order, cutoff_frequency / (sample_rate / 2), btype='low')
    filtered_data = lfilter(b, a, data)
    return filtered_data

read_rate, read_audio_data = wavfile.read(raw_output_file)
print(read_rate)

filtered_audio = low_pass_filter(read_audio_data, CUTTOFF_FREQUENCY, read_rate)

# Apply the high-pass filter to our audio data
# filtered_audio = high_pass_filter(read_audio_data, 16000, read_rate)


wavfile.write(filtered_output_file, read_rate, filtered_audio)

44064


# That's it! It might sound bad, but it is converted